
# Chat Security

This article addresses the development of an information security chatbot through fine-tuning of the Llama (open-source language model). It employs the QLora methodology to adjust the model's weights by retraining them using a database comprised of questions and answers related to information security. The model outperformed the Llama-7B model in Portuguese tasks overall, excelling particularly in Semantic Similarity and Textual Entailment activities. In question-answering tasks, our project approached the average performance of the Sabiá-7B model.

In [5]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard openpyxl scipy boto3

In [14]:
# Imports
import os
import torch
import peft
import trl
import datasets
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging,
                          TextStreamer)
import warnings
warnings.filterwarnings('ignore')
logging.set_verbosity(logging.CRITICAL)

import torch
torch.cuda.empty_cache()

TypeError: expected string or bytes-like object

In [ ]:
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

## Definindo Parâmetros do Ajuste Fino

In [4]:
import pandas as pd
df = pd.read_excel('base_dados_segurança.xlsx')


df['finetuning'] = df.apply(lambda x: '<s>[INST]' + x['PERGUNTAS'] + ' [/INST] ' + x['RESPOTAS'] + '<\s> \n', axis=1)
df['finetuning'].to_csv('data.txt', sep='\t', index=False)


df = df.sample(frac=1).reset_index(drop=True)

In [5]:
dataset = load_dataset("text", data_files="data.txt",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
#Quantização

use_4bit = True # Ativa precisão 4-bits ao carregar o modelo base
bnb_4bit_compute_dtype = "float16" # Define o dtype para os parâmetros do modelo base
bnb_4bit_quant_type = "nf4" # NF4 = Normal Float 4
use_nested_quant = False # Desativa a quantização aninhada para modelos básicos de 4 bits (quantização dupla)

# Carrega o tokenizer e modelo com configuração QLoRA
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

## Carregando o Modelo Base e o Tokenizador

In [7]:
# Nome do modelo BASE
modelo_base = "NousResearch/Llama-2-7b-chat-hf"
modelo = AutoModelForCausalLM.from_pretrained(modelo_base,quantization_config = bnb_config,device_map = {"": 0})
modelo.config.use_cache = False
modelo.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [8]:
# Carrega o tokenizador do LLaMA2
tokenizer = AutoTokenizer.from_pretrained(modelo_base, trust_remote_code = True,skip_special_tokens=True, use_auth_token=access_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## Configurando dados de Treinamento e Ajuste Fino

In [9]:
# LORA
lora_r = 64 # Dimensão da atenção LoRA
lora_alpha = 16 # Parâmetro Alpha para LoRA scaling
lora_dropout = 0.1 # Probabilidade Dropout para as camadas LoRA
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

In [10]:
batch_treinamento = 20
epochs = 10
optim = "paged_adamw_32bit"
max_grad_norm = 0.3 # Maximum gradient normal (gradient clipping)
learning_rate = 2e-4
weight_decay = 0.001 # Redução de peso a ser aplicada a todas as camadas, exceto pesos bias/LayerNorm
lr_scheduler_type = "cosine" # Learning rate schedule
warmup_ratio = 0.03 # Proporção de passos para um aquecimento linear (de 0 à taxa de aprendizagem)

# Habilita o treinamento fp16/bf16
fp16 = False
bf16 = True

In [11]:
# Parâmetros de configuração de treino
training_arguments = TrainingArguments(output_dir = "resultados",
                                       num_train_epochs = epochs,
                                       per_device_train_batch_size = batch_treinamento,
                                       gradient_accumulation_steps = 1, # Número de etapas de atualização para acumular os gradientes
                                       optim = optim,
                                       save_steps = 0,
                                       logging_steps = 25,  # Registrar cada X etapas de atualizações
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = fp16,
                                       bf16 = bf16,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = -1, # Número de etapas de treinamento
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = True, # Agrupar sequências em lotes com o mesmo comprimento
                                       lr_scheduler_type = lr_scheduler_type,
                                       report_to = "tensorboard")

In [12]:
# Definir parâmetros de ajuste fino com SFTTrainer
trainer = SFTTrainer(model = modelo,
                     train_dataset = dataset,
                     peft_config = peft_config,
                     dataset_text_field = "text",
                     max_seq_length = None, #Comprimento máximo de sequência a ser usado
                     tokenizer = tokenizer,
                     args = training_arguments,
                     packing = False) # Reunir vários exemplos curtos na mesma sequência de entrada para aumentar a eficiência

Map:   0%|          | 0/1686 [00:00<?, ? examples/s]

## Treinamento e Ajuste Fino

In [ ]:
# Treina o modelo
trainer.train()

{'loss': 3.6395, 'learning_rate': 0.00019998159678028148, 'epoch': 0.15}
{'loss': 4.0737, 'learning_rate': 0.0001990342096935135, 'epoch': 0.3}
{'loss': 0.7401, 'learning_rate': 0.00019666461831457439, 'epoch': 0.44}
{'loss': 0.6302, 'learning_rate': 0.0001929068517233169, 'epoch': 0.59}
{'loss': 2.5337, 'learning_rate': 0.00018781487421946734, 'epoch': 0.74}
{'loss': 0.9528, 'learning_rate': 0.00018146181035548113, 'epoch': 0.89}
{'loss': 0.2963, 'learning_rate': 0.00017393889481403784, 'epoch': 1.04}
{'loss': 0.6428, 'learning_rate': 0.000165354162210709, 'epoch': 1.18}
{'loss': 0.4539, 'learning_rate': 0.00015583089563719985, 'epoch': 1.33}
{'loss': 0.6336, 'learning_rate': 0.00014550585622522693, 'epoch': 1.48}
{'loss': 0.7579, 'learning_rate': 0.00013452731915580075, 'epoch': 1.63}
{'loss': 1.881, 'learning_rate': 0.0001230529443182689, 'epoch': 1.78}
{'loss': 0.7043, 'learning_rate': 0.00011124751219802545, 'epoch': 1.92}
{'loss': 0.3803, 'learning_rate': 9.92805575072098e-05, 'e

In [ ]:
# Salva o modelo ajustado
trainer.save_model("model/sectrum_model")

In [ ]:
modelo.config.to_json_file("config.json")

## Gerando Texto com o Modelo Ajustado

In [ ]:
%%time
# Prepara o prompt
prompt = "Como proteger meu endpoint?"

# Cria o pipeline
pipe = pipeline(task = "text-generation",
                model = modelo,
                tokenizer = tokenizer,
                max_length = 300,
                streamer=TextStreamer(tokenizer,skip_prompt=True))


# Executa o pipeline e gera o texto a partir do prompt inicial
pipe(f"[INST] {prompt} [/INST]")[0]['generated_text']

